In [26]:
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
from google import genai
from tavily import TavilyClient
import os
import re

In [28]:
# Create client
client = genai.Client(api_key=os.environ.get('GOOGLE_API_KEY'))
model="gemini-2.5-pro"
tavily_client = TavilyClient(api_key=os.environ.get('TAVILY_API_KEY')) # Initialize Tavily client

In [29]:
#Agents
class ProductManagerAgent:
    """Product Manager Agent with conversation memory."""
    
    def __init__(self, model: str, client):
        self.model = model
        self.client = client
        self.conversation_history = []
        self.system_instruction = """You are an experienced Product Manager with expertise in:
        - Product strategy and roadmap planning
        - Market research and competitive analysis
        - User research and requirements gathering
        - Feature prioritization and backlog management
        - Stakeholder communication
        - Metrics and KPI definition
        - Go-to-market strategies
        - Agile methodologies
        - Providing feedback to your product specialist, engineering, design, data science, testing, and marketing teams

        You provide practical, actionable advice and ask clarifying questions when needed.
        When provided with search results, incorporate relevant information into your response and cite sources."""
    
    def __call__(self, user_query):
        """Make the agent callable like a function."""
        # First, ask the model if it needs to search
        decision_prompt = f"""You are a Product Manager agent. Analyze this query and determine if you need to search the web for current information.

Query: {user_query}

Respond with ONLY "YES" if the query requires:
- Recent market data, trends, or statistics
- Current product information or pricing
- Recent news or events
- Up-to-date best practices or tools
- Competitive analysis requiring current data

Respond with ONLY "NO" if the query is about:
- General product management principles
- Timeless frameworks and methodologies
- Hypothetical scenarios
- Your expertise and advice

Response (YES or NO):"""

        decision_response = self.client.models.generate_content(
            model=self.model,
            contents=decision_prompt
        )
        
        needs_search = "YES" in decision_response.text.strip().upper()
        
        # Perform search if needed
        context = ""
        if needs_search:
            print("🔍 Searching the web for current information...")
            search_results = search_web(user_query)
            if isinstance(search_results, list):
                context = "\n\nWeb Search Results:\n"
                for i, result in enumerate(search_results, 1):
                    context += f"\n{i}. {result.get('title', '')}\n{result.get('content', '')}\nURL: {result.get('url', '')}\n"
        
        # Build conversation context
        conversation_context = ""
        if self.conversation_history:
            conversation_context = "\n\nPrevious conversation:\n"
            for entry in self.conversation_history[-5:]:  # Keep last 5 exchanges
                conversation_context += f"User: {entry['user']}\nAssistant: {entry['assistant']}\n\n"
        
        full_query = conversation_context + "Current query: " + user_query + context
        
        response = self.client.models.generate_content(
            model=self.model,
            contents=full_query,
            config={'system_instruction': self.system_instruction}
        )
        
        # Store in conversation history
        self.conversation_history.append({
            'user': user_query,
            'assistant': response.text
        })
        
        return response.text
    
    def clear_history(self):
        """Clear conversation history."""
        self.conversation_history = []
        print("Conversation history cleared.")





In [30]:
#Helper functions
def remove_markdown(response_text):
    """Remove markdown formatting and print clean text."""
    # Remove markdown formatting
    text = re.sub(r'\*\*(.+?)\*\*', r'\1', response_text)  # Bold
    text = re.sub(r'\*(.+?)\*', r'\1', text)  # Italic
    text = re.sub(r'`(.+?)`', r'\1', text)  # Code
    text = re.sub(r'#+\s', '', text)  # Headers
    return text
def chat_with_agent(agent, agent_name="Agent"):
    """
    Generic interactive chat interface for any agent.
    
    Args:
        agent: The agent instance (callable) to chat with
        agent_name: Display name for the agent (default: "Agent")
    
    Type 'exit', 'quit', or 'bye' to end the conversation.
    Type 'clear' to clear conversation history.
    """
    print("=" * 60)
    print(f"{agent_name} Chat")
    print("=" * 60)
    print(f"Start chatting with the {agent_name}!")
    print("Type 'exit', 'quit', or 'bye' to end the chat.")
    print("Type 'clear' to clear conversation history.\n")
    
    while True:
        user_input = input("You: ").strip()
        
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print(f"\n{agent_name}: Thanks for chatting! Goodbye!")
            break
        
        if user_input.lower() == 'clear':
            if hasattr(agent, 'clear_history'):
                agent.clear_history()
            continue
            
        if not user_input:
            continue
            
        try:
            response = agent(user_input)
            print(f"\n{agent_name}: {remove_markdown(response)}\n")
        except Exception as e:
            print(f"\nError: {e}\n")
            print("Please try again or type 'exit' to quit.\n")
def search_web(query):
    """Search the web using Tavily."""
    try:
        response = tavily_client.search(query, max_results=5)
        return response['results']
    except Exception as e:
        return f"Search error: {e}"



In [31]:
# Create agent instances
pm_agent = ProductManagerAgent(model=model, client=client)

In [35]:
# Test the agent
query = "What was the last question I asked you?"
response = pm_agent(query)
print(remove_markdown(response))

That's a great way to test the system!

As we established in our previous exchange, I don't retain memory of past conversations. Think of each interaction with me as a new product discovery session – we start with a clean slate, ready to tackle the problem at hand.

So, for all practical purposes, the last question you asked me is "What was the last question I asked you?"

Now, let's pivot to the work. What's the most pressing product challenge we need to solve today? Are we:

*   Defining a Q3 roadmap?
*   Running a competitive analysis on a new market entrant?
*   Prioritizing features for the next sprint?
*   Prepping a go-to-market plan for a major launch?

Let me know what's top of mind, and we can dive in.
